In [1]:
from owlready2 import get_ontology

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [3]:
import nilearn
atlas_destrieux = nilearn.datasets.fetch_atlas_destrieux_2009()

/Users/gzanitti/miniconda3/envs/neurolang/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
/Users/gzanitti/miniconda3/envs/neurolang/lib/python3.7/site-packages/numpy/lib/npyio.py:2322: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  output = genfromtxt(fname, **kwargs)


In [2]:
onto = get_ontology('./fma.owl')
onto.load()

get_ontology("http://purl.org/sig/ont/fma.owl#")

In [3]:
'''
def get_nodes(onto, main_label, labels=[]):
    for main in onto.search(label = main_label):
        #import ipdb; ipdb.set_trace()
        for sub in main.subclasses():
            labels = get_members(onto, sub.label, labels)
        labels.append(main.label[0])
    return labels 

def get_subclasses(onto, main_label, subclasses=dict({})):
    for main in onto.search(label = main_label):
        subs = []
        for sub in main.subclasses():
            subs.append(sub.label[0])
            subclasses = get_subclasses(onto, sub.label, subclasses)
        if len(subs) > 0:
            subclasses[main.label[0]] = subs
    return subclasses 
'''

'\ndef get_nodes(onto, main_label, labels=[]):\n    for main in onto.search(label = main_label):\n        #import ipdb; ipdb.set_trace()\n        for sub in main.subclasses():\n            labels = get_members(onto, sub.label, labels)\n        labels.append(main.label[0])\n    return labels \n\ndef get_subclasses(onto, main_label, subclasses=dict({})):\n    for main in onto.search(label = main_label):\n        subs = []\n        for sub in main.subclasses():\n            subs.append(sub.label[0])\n            subclasses = get_subclasses(onto, sub.label, subclasses)\n        if len(subs) > 0:\n            subclasses[main.label[0]] = subs\n    return subclasses \n'

In [37]:
def get_nodes(onto, main_label, labels=[]):
    for main in onto.search(label = main_label):
        subs = list(main.subclasses())
        while subs:
            subc = subs.pop(0)
            labels.append(subc.label[0])
            subs = subs + list(subc.subclasses())
    labels = list(dict.fromkeys(labels))
    return labels

def get_subclasses(onto, main_label, apply=None):
    subclasses = dict()
    for main in onto.search(label = main_label):
        subs = [main]
        while subs:
            subc = subs.pop(0)
            temp = list(subc.subclasses())
            if len(temp) > 0:
                subclasses[subc.label[0]] = [e.label[0] for e in temp]
                subs = subs + temp
    if apply is not None:
        subclasses = apply(subclasses)
    return subclasses

def parse_isSub(subc):
    subclass = []
    for k, v in subc.items():
        for elem in v:
            subclass.append((k, elem))
    return subclass

def get_synonyms(onto, main_label, apply=None):
    synonyms = dict()
    for main in onto.search(label = main_label):
        subs = [main]
        while subs:
            subc = subs.pop(0)
            temp = list(subc.subclasses())
            syn = subc.synonym
            if len(syn) > 0:
                synonyms[subc.label[0]] = syn
            subs = subs + temp
    if apply is not None:
        synonyms = apply(synonyms)
    return synonyms

def parse_isSyn(syn):
    syns = []
    for k, v in syn.items():
        for elem in v:
            syns.append((k, elem))
            syns.append((elem, k))
    return syns

In [38]:
#nodes = get_nodes(onto, 'Anatomical entity')
#subc = get_subclasses(onto, 'Anatomical entity', apply=parse_isSub)
#syn = get_synonyms(onto, 'Anatomical entity', apply=parse_isSyn)

In [39]:
from neurolang import frontend as fe
dl = fe.NeurolangDL()

nodes = get_nodes(onto, 'Anatomical entity')
subc = get_subclasses(onto, 'Anatomical entity', apply=parse_isSub)
syn = get_synonyms(onto, 'Anatomical entity', apply=parse_isSyn)

dl.add_tuple_set(
    ((str(e),) for e in nodes),
    name='nodes'
);

dl.add_tuple_set(
    ((str(e1), str(e2),) for e1, e2 in subc),
    name='subclasses'
);

dl.add_tuple_set(
    ((str(e1), str(e2),) for e1, e2 in syn),
    name='synonyms'
);

In [58]:
x = dl.new_symbol(name='x')
y = dl.new_symbol(name='y')
val = dl.new_symbol(name='val')
isSub = dl.new_symbol(name='isSub')
isSyn = dl.new_symbol(name='isSyn')

query1 = dl.query(val(x), dl.symbols.nodes(x))
query2 = dl.query(isSyn(x, y), dl.symbols.synonyms(x, y))
query3 = dl.query(isSub(x, y), dl.symbols.subclasses(x, y))

In [60]:
dl.query(val(y), isSub('Lobe of cerebral hemisphere', y))

val: typing.AbstractSet[typing.Tuple[str]] = [('Frontal lobe',), ('Temporal lobe',), ('Parietal lobe',), ('Occipital lobe',), ('Insula',), ('Limbic lobe',)]

In [61]:
dl.query(val(y), isSyn('Lobe of cerebral hemisphere', y))

val: typing.AbstractSet[typing.Tuple[str]] = [('Cerebral hemisphere lobe',), ('Cerebral lobe',)]